In [1]:
import skimage as ski
import numpy as np
from skimage.util import invert
from skimage.color import rgb2gray
%matplotlib inline
import matplotlib.pyplot as plt
import os
from skimage.morphology import skeletonize, thin
import cv2

filename = 'ATTA_GNS.png'
image = ski.io.imread(filename)
image = rgb2gray(image)
for i in range(50):
    image = ski.filters.gaussian(image)
""" further enhancement an be made with opening and closing the image
background can be worked out suing selective backgrounds (source youtube Nicolai)"""
copied = image.copy()
print(np.shape(copied))
for r in range(4096):
    for i in range(4096):
        a = round(image[r][i],2)
        if a > 0.6:
            copied[r][i]= 1
        else:
            copied[r][i] = 0
#plt.imshow(copied, cmap = 'gray')
image = invert(copied)
skeleton = skeletonize(image)
''' can be enhanced by opening and closing to get rid of small curves
  the centres can be found by open function only on the core (core is colored differently)'''
thinned_partial = thin(image, max_num_iter=40)

fig, axes = plt.subplots(1,3, figsize=(8, 8), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(skeleton, cmap=plt.cm.gray)
ax[0].set_title('skeleton')
ax[0].axis('off')

ax[1].imshow(thinned_partial, cmap=plt.cm.gray)
ax[1].set_title('partially thinned')
ax[1].axis('off')

ax[2].imshow(invert(image), cmap=plt.cm.gray)
ax[2].set_title('original')
ax[2].axis('off')


fig.tight_layout()
plt.show()

FileNotFoundError: No such file: 'c:\Users\muham\vs_code\AMAP\ATTA_GNS.png'

In [2]:
image = thinned_partial.copy()
image = image.astype(np.float32)
for i in range(100):
    image = ski.filters.gaussian(image)
copied = image.copy()
print(np.shape(copied))
for r in range(4096):
    for i in range(4096):
        a = round(image[r][i],2)
        if a > 0.6:
            copied[r][i]= 1
        else:
            copied[r][i] = 0
''' instead of using Gaussian blur and thresholding everything, eroding can be more efficient
 for later dilation '''
thinned_partial = thinned_partial.astype(np.uint8)
cores = cv2.bitwise_and(copied,copied, mask= invert(thinned_partial))
plt.imshow(cores, 'gray')

NameError: name 'thinned_partial' is not defined

In [3]:
lines = skeleton.copy()
'''
here parallel computation will be more efficient

'''
for r in range(4096):
    for i in range(4096):
        a = skeleton[r][i] - cores[r][i]
        if a > 0:
            lines[r][i]= 1
        else:
            lines[r][i] = 0
plt.imshow(lines,'gray')

NameError: name 'skeleton' is not defined

In [4]:
'''
dilation

'''
# out = lines.copy()
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(150,150),(1,-1))
# cv2.dilate(lines,out,kernel,(-1,-1),1)
# plt.imshow(out,'gray')
# image = lines
# scaled_image = (image + np.min(image)) / (np.max(image) + np.min(image)) * 255
# image_uint8 = scaled_image.astype(np.uint8)
# ski.io.imsave('outputh.png', image_uint8)

'\ndilation\n\n'

In [5]:
lines = lines.astype(np.uint8)
out = np.zeros(np.shape(lines))
edged = cv2.Canny(lines, 1, 1) 
contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
print(len(contours))
size = []
out = cv2.drawContours(out, contours,-1,1,3)
image = out
scaled_image = (image + np.min(image)) / (np.max(image) + np.min(image)) * 255
image_uint8 = scaled_image.astype(np.uint8)
ski.io.imsave('outputh.png', image_uint8)

NameError: name 'lines' is not defined

In [6]:
image = ski.io.imread('out.png', image_uint8)
out = np.zeros(np.shape(lines))
edged = cv2.Canny(image, 1, 1) 
contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
print(len(contours))
aprx = []
for i in range(len(contours)):
    #print(f'{len(contours[i])} {i}')
    length = cv2.arcLength(contours[i],0)
    aprx.append(cv2.approxPolyDP(contours[i],20, True))
    #print(f'{len(aprx[i])} {i}')
    #size.append(length)
    #cv2.putText(out,str(round(length)),contours[i][0][0],2,1,1,2) # 1,2) first is the color
for i in range(len(aprx)):
    (x,y), radius = cv2.minEnclosingCircle(aprx[i])
    length = cv2.arcLength(aprx[i],0)
    cv2.putText(out,str(round(length)),(int(x)+3,int(y)+3),1,2,1,2)
cv2.drawContours(out, aprx,-1,1,3)
plt.imshow(out,'gray')

NameError: name 'image_uint8' is not defined

In [7]:
image = out
scaled_image = (image + np.min(image)) / (np.max(image) + np.min(image)) * 255
image_uint8 = scaled_image.astype(np.uint8)
ski.io.imsave('outputh.png', image_uint8)

NameError: name 'out' is not defined

In [8]:
center, radius = cv2.minEnclosingCircle(aprx[0])
print(center)

NameError: name 'aprx' is not defined

In [9]:
''' 
ML to detect the shoulders of the spike

Go ahead with perfecting the segmentation
Work on all the pictures
take up the lining
take up the C++ (or even go with Python for time being)
follow the line and hack it

take the middle centres and go in all directions and find the branches

k-means -> taking up the skeleton for similar pixels?

'''

' \nML to detect the shoulders of the spike\n\nGo ahead with perfecting the segmentation\nWork on all the pictures\ntake up the lining\ntake up the C++ (or even go with Python for time being)\nfollow the line and hack it\n\ntake the middle centres and go in all directions and find the branches\n\nk-means -> taking up the skeleton for similar pixels?\n\n'